# Binary CSP solver

*Guillaume DALLE*

## 1. CSP structure and backtracking

### 1.1) CSP object

In [1]:
mutable struct CSP
    n_variables::Int64
    names::Array{String, 1}
    domain::Array{Array{Int64, 1}, 1}
    domain_max::Array{Int64, 1}
        
    n_constraints::Int64
    constraint_var1::Array{Int64, 1}
    constraint_var2::Array{Int64, 1}
    matrix_storage::Bool
    constraint_satisfaction_mat::Array{Array{Bool, 2}, 1}
    constraint_satisfaction_fun::Array{Function, 1}
    
    var1_constraints::Array{Array{Int64, 1}, 1}
    var2_constraints::Array{Array{Int64, 1}, 1}
    
    backtracking_params::Dict{String, Any}
    
    CSP() = new(
        0,
        String[],
        Array{Int64, 1}[],
        Int64[],
        
        0,
        Int64[],
        Int64[],
        false,
        Array{Bool, 2}[],
        Function[],
        
        Array{Int64, 1}[],
        Array{Int64, 1}[],
        
        Dict{String, Any}(
            "matrix_storage"=>false,
            "variable_selection"=>"domain_size",
            "look_ahead_method"=>"FC",
            "time_limit"=>10.,
        ),
    )
end

### 1.2) Variables and constraints

In [2]:
function add_variable!(csp::CSP, name::String, domain_size::Int64)
    csp.n_variables += 1
    push!(csp.names, name)
    push!(csp.domain, collect(1:domain_size))
    push!(csp.domain_max, domain_size)
    
    push!(csp.var1_constraints, Int64[])
    push!(csp.var2_constraints, Int64[])
end

add_variable! (generic function with 1 method)

In [3]:
function add_individual_constraint!(csp::CSP, var::Int64, individual_feasibility::Function)
    i = 1
    while i <= csp.domain_max[var]
        value = csp.domain[var][i]
        if !individual_feasibility(csp, var, value)
            csp.domain[var][i] = csp.domain[var][csp.domain_max[var]]
            csp.domain[var][csp.domain_max[var]] = value
            csp.domain_max[var] -= 1
        else
            i += 1
        end
    end
end

add_individual_constraint! (generic function with 1 method)

In [4]:
function add_constraint!(csp::CSP, var1::Int64, var2::Int64, compatibility::Function)
    csp.n_constraints += 1
    
    push!(csp.constraint_var1, var1)
    push!(csp.constraint_var2, var2)
    push!(csp.var1_constraints[var1], csp.n_constraints)
    push!(csp.var2_constraints[var2], csp.n_constraints)

    push!(csp.constraint_satisfaction_fun, compatibility)
end

add_constraint! (generic function with 1 method)

In [5]:
function set_backtracking_params(csp::CSP, backtracking_params::Dict{String, Any})
    for (key, val) in backtracking_params
        csp.backtracking_params[key] = val
    end
end

set_backtracking_params (generic function with 1 method)

### 1.3) Consistency check

In [6]:
function check_feasibility(csp::CSP, instantiation::Array{Int64, 1}, new_var::Int64)
    if new_var == -1
        return true
    end
    for cons in csp.var1_constraints[new_var]
        var2::Int64 = csp.constraint_var2[cons]
        value1 = instantiation[new_var]
        value2 = instantiation[var2]
        if (value1 !== -1) && (value2 !== -1)
            if csp.matrix_storage
                satisfied = csp.constraint_satisfaction_mat[cons][value1, value2]
            else
                satisfied = csp.constraint_satisfaction_fun[cons](csp, new_var, var2, value1, value2)
            end
            if !satisfied
                return false
            end
        end
    end
    return true
end

check_feasibility (generic function with 1 method)

### 1.4) Choice of variable / value

In [7]:
function evaluate_variable(csp::CSP, var::Int64)
    if csp.backtracking_params["variable_selection"] == "domain_size"
        eval = - csp.domain_max[var]
    elseif csp.backtracking_params["variable_selection"] == "domain_size_semirandom"
        eval = - csp.domain_max[var] + abs(var - csp.n_variables / 2) / csp.n_variables
    elseif csp.backtracking_params["variable_selection"] == "domain_size_random"
        eval = - csp.domain_max[var] + rand()
    elseif csp.backtracking_params["variable_selection"] == "constraints"
        eval = length(csp.var1_constraints[var])
    elseif csp.backtracking_params["variable_selection"] == "constraints_random"
        eval = length(csp.var1_constraints[var]) + rand()
    elseif csp.backtracking_params["variable_selection"] == "random"
        eval = rand()
    else
        eval = 0
    end
    return eval
end

function choose_next_variable(csp::CSP, instantiation::Array{Int64, 1})
    best_var::Int64 = -1
    best_eval::Float64 = -Inf
        
    for var in 1:csp.n_variables
        if instantiation[var] != -1
            continue
        else
            eval = evaluate_variable(csp, var)
            if eval > best_eval
                best_var = var
                best_eval = eval
            end
        end
    end
    return best_var
end

choose_next_variable (generic function with 1 method)

### 1.5) Look-ahead

In [8]:
function do_look_ahead!(csp::CSP, instantiation::Array{Int64, 1}, x::Int64)
    previous_domain_max::Array{Int64, 1} = copy(csp.domain_max)
    
    a::Int64 = instantiation[x]
    for i in 1:csp.domain_max[x]
        if (csp.domain[x][i] == a)
            csp.domain[x][i] = csp.domain[x][1]
            csp.domain[x][1] = a
            csp.domain_max[x] = 1
            break
        end
    end
    
    if csp.backtracking_params["look_ahead_method"] == "FC"
        forward_checking!(csp, instantiation, x)
    elseif csp.backtracking_params["look_ahead_method"] == "MAC3"
        AC3!(csp, instantiation)
    end
    
    return previous_domain_max
end

function undo_look_ahead!(csp::CSP, previous_domain_max::Array{Int64, 1})
    csp.domain_max = previous_domain_max
end

undo_look_ahead! (generic function with 1 method)

In [9]:
function forward_checking!(csp::CSP, instantiation::Array{Int64, 1}, x::Int64)
    
    a::Int64 = instantiation[x]
    
    for cons in csp.var1_constraints[x]
        y = csp.constraint_var2[cons]
        if instantiation[y] != -1
            continue
        end

        j = 1
        while j <= csp.domain_max[y]
            b = csp.domain[y][j]
            if csp.matrix_storage
                satisfied = csp.constraint_satisfaction_mat[cons][a, b]
            else
                satisfied = csp.constraint_satisfaction_fun[cons](csp, x, y, a, b)
            end
            if !satisfied
                csp.domain[y][j] = csp.domain[y][csp.domain_max[y]]
                csp.domain[y][csp.domain_max[y]] = b
                csp.domain_max[y] -= 1
            else
                j += 1
            end
        end
    end
end

forward_checking! (generic function with 1 method)

In [10]:
function AC3!(csp::CSP, instantiation::Array{Int64, 1})
    
    to_test = Set{Int64}()
    for cons in 1:csp.n_constraints
        push!(to_test, cons)
    end
    
    while !isempty(to_test)
        cons = pop!(to_test)
        x = csp.constraint_var1[cons]
        y = csp.constraint_var2[cons]
        
        i = 1
        while i <= csp.domain_max[x]
            a = csp.domain[x][i]
            a_supported::Bool = false
            for b in csp.domain[y][1:csp.domain_max[y]]
                if csp.matrix_storage
                    satisfied = csp.constraint_satisfaction_mat[cons][a, b]
                else
                    satisfied = csp.constraint_satisfaction_fun[cons](csp, x, y, a, b)
                end
                if satisfied
                    a_supported = true
                    break
                end
            end
            
            if !a_supported
                csp.domain[x][i] = csp.domain[x][csp.domain_max[x]]
                csp.domain[x][csp.domain_max[x]] = a
                csp.domain_max[x] -= 1
                for cons_impacted in csp.var2_constraints[x]
                    if csp.constraint_var1[cons] != y
                        push!(to_test, cons_impacted)
                    end
                end
            else
                i += 1
            end
        end
    end

end

AC3! (generic function with 1 method)

### 1.6) Backtracking

In [11]:
function backtrack!(
        csp::CSP,
        instantiation::Array{Int64, 1},
        new_var::Int64,
        nodes_explored::Int64,
        initial_time::Float64
    )
    if time() - initial_time > csp.backtracking_params["time_limit"]
        error("Time limit exceeded")
    end
    if !check_feasibility(csp, instantiation, new_var)
        return (false, instantiation, nodes_explored)
    end
    
    new_var = choose_next_variable(csp, instantiation)
    if new_var == -1
        return true, instantiation, nodes_explored
    end
        
    domain_to_explore = copy(csp.domain[new_var][1:csp.domain_max[new_var]])
    for new_value in domain_to_explore
        nodes_explored += 1
        instantiation[new_var] = new_value
        look_ahead_result = do_look_ahead!(csp, instantiation, new_var)
                
        (solution_found, solution, nodes_explored) = backtrack!(
            csp, instantiation, new_var, nodes_explored, initial_time)
        if solution_found
            return (true, solution, nodes_explored)
        end
        
        instantiation[new_var] = -1
        undo_look_ahead!(csp, look_ahead_result)

    end
    return false, instantiation, nodes_explored
end

backtrack! (generic function with 1 method)

In [12]:
function add_matrix_storage!(csp::CSP)
    csp.matrix_storage = true
    for cons in 1:csp.n_constraints
        var1 = csp.constraint_var1[cons]
        var2 = csp.constraint_var2[cons]
        satisfaction = zeros(Bool, length(csp.domain[var1]), length(csp.domain[var2]))
        for value1 in csp.domain[var1], value2 in csp.domain[var2]
            if csp.constraint_satisfaction_fun[cons](csp, var1, var2, value1, value2)
                satisfaction[value1, value2] = true
            end
        end
        push!(csp.constraint_satisfaction_mat, satisfaction)
    end
end

function preprocess!(csp::CSP)
    if csp.backtracking_params["matrix_storage"]
        add_matrix_storage!(csp)
    end
end

preprocess! (generic function with 1 method)

In [13]:
function solve!(csp::CSP)
    preprocess!(csp)
    return backtrack!(csp, -ones(Int64, csp.n_variables), -1, 0, time())
end

function solve_verbose!(csp::CSP)
    (status, instantiation, nodes_explored) = solve!(csp)
    if !status
        println("\nStatus: unfeasible")
    else
        println("\nStatus: solved")
        println("\nNodes explored: ", nodes_explored)
        println("\nSolution: ")
        for var in 1:csp.n_variables
            println(csp.names[var], ": ", instantiation[var])
        end
    end
    println()
    return (status, instantiation, nodes_explored)
end

solve_verbose! (generic function with 1 method)

## 2. Applications

### 2.1) N-queens problem

In [14]:
function compatibility_nqueens(csp::CSP, var1::Int64, var2::Int64, value1::Int64, value2::Int64)
    # Exclude same row, same diagonal, same antidiagonal
    return (value1 != value2) && abs(value1 - value2) != abs(var1 - var2)
end

function define_nqueens(n::Int64)
    csp = CSP()
    for i in 1:n
        name = "row_" * string(i)
        domain_size = n
        add_variable!(csp, name, domain_size)
    end
    # Binary constraints
    for i in 1:n, j in 1:n
        if (i != j)
            add_constraint!(csp, i, j, compatibility_nqueens)
        end
    end
    return csp
end

function visualize_nqueens(csp::CSP, instantiation::Array{Int64, 1})
    instantiation_dict::Dict{String, Int64} = Dict(
        csp.names[var] => instantiation[var] for var in 1:csp.n_variables)
    n::Int64 = length(instantiation_dict)
    for j in 1:n
        println()
        for i in 1:n
            if instantiation_dict["row_" * string(j)] == i
                print(" Q")
            else
                print(" .")
            end
        end
    end
end

function check_instantiation_nqueens(csp::CSP, instantiation::Array{Int64, 1})
    n::Int64 = length(instantiation)
    for var1 in 1:n, var2 in 1:var1-1
        if !compatibility_nqueens(csp, var1, var2, instantiation[var1], instantiation[var2])
            return false
        end
    end
    return true
end

check_instantiation_nqueens (generic function with 1 method)

In [15]:
nqueens_big = define_nqueens(400);
set_backtracking_params(
    nqueens_big,
    Dict(
        "matrix_storage"=>false,
        "time_limit"=>1.,
        "look_ahead_method"=> "FC",
        "variable_selection"=> "domain_size_random"
    )
)
@time b, i, n = solve!(nqueens_big)

  0.696150 seconds (474.32 k allocations: 25.518 MiB)


(true, [224, 296, 275, 273, 236, 210, 254, 247, 241, 226  …  278, 56, 90, 113, 188, 186, 112, 39, 76, 343], 449)

In [16]:
nqueens_small = define_nqueens(10)
b, i, n = solve_verbose!(nqueens_small)
visualize_nqueens(nqueens_small, i)


Status: solved

Nodes explored: 81

Solution: 
row_1: 1
row_2: 10
row_3: 7
row_4: 5
row_5: 8
row_6: 2
row_7: 9
row_8: 3
row_9: 6
row_10: 4


 Q . . . . . . . . .
 . . . . . . . . . Q
 . . . . . . Q . . .
 . . . . Q . . . . .
 . . . . . . . Q . .
 . Q . . . . . . . .
 . . . . . . . . Q .
 . . Q . . . . . . .
 . . . . . Q . . . .
 . . . Q . . . . . .

### 2.2) Knight's tour

In [17]:
function coord_from_square(square::Int64, board_size::Int64)
    row = div(square - 1, board_size) + 1
    col = rem(square - 1, board_size) + 1
    return (row, col)
end

function square_from_coord(row::Int64, col::Int64, board_size::Int64)
    square = (row-1) * board_size + col
    return square
end

square_from_coord (generic function with 1 method)

In [18]:
function rook_move(square1::Int64, square2::Int64, n::Int64)
    row1, col1 = coord_from_square(square1, n)
    row2, col2 = coord_from_square(square2, n)
    return (square1 != square2) && ((row1 == row2) || (col1 == col2))
end

function bishop_move(square1::Int64, square2::Int64, n::Int64)
    row1, col1 = coord_from_square(square1, n)
    row2, col2 = coord_from_square(square2, n)
    return (square1 != square2) && (abs(row1 - row2) == abs(col1 - col2))
end

function knight_move(square1::Int64, square2::Int64, n::Int64)
    row1, col1 = coord_from_square(square1, n)
    row2, col2 = coord_from_square(square2, n)
    return (
        (abs(row1 - row2) == 1 && abs(col1 - col2) == 2) ||
        (abs(row1 - row2) == 2 && abs(col1 - col2) == 1)
    )
end

function queen_move(square1::Int64, square2::Int64, n::Int64)
    return rook_move(square1, square2, n) || bishop_move(square1, square2, n)
end

function king_move(square1::Int64, square2::Int64, n::Int64)
    row1, col1 = coord_from_square(square1, n)
    row2, col2 = coord_from_square(square2, n)
    return (square1 != square2) && (max(abs(row1 - row2), abs(col1 - col2)) == 1)
end

function piece_move(square1::Int64, square2::Int64, n::Int64, piece::String)
    if piece == "knight"
        return knight_move(square1, square2, n)
    elseif piece == "bishop"
        return bishop_move(square1, square2, n)
    elseif piece == "rook"
        return rook_move(square1, square2, n)
    elseif piece == "queen"
        return queen_move(square1, square2, n)
    elseif piece == "king"
        return king_move(square1, square2, n)
    end
end

piece_move (generic function with 1 method)

In [19]:
function compatibility_tour(
        csp::CSP,
        var1::Int64, var2::Int64,
        value1::Int64, value2::Int64,
        n::Int64, piece::String
    )
    if abs(var1 - var2) > 1
        return value1 != value2
    else
        return piece_move(value1, value2, n, piece)
    end
end

function define_tour(n::Int64, piece::String)
    csp = CSP()
    if piece == "bishop"
        n_pieces = div(n^2, 2)
    else
        n_pieces = n^2
    end
    for i in 1:n_pieces
        name = "position_" * string(i)
        domain_size = n^2
        add_variable!(csp, name, domain_size)
    end
    for i in 1:n_pieces, j in 1:n_pieces
        if (i != j)
            compatibility_tour_piece(a, b, c, d, e) = compatibility_tour(a, b, c, d, e, n, piece)
            add_constraint!(csp, i, j, compatibility_tour_piece)
        end
    end
    return csp
end

function visualize_tour(csp::CSP, instantiation::Array{Int64, 1}, n::Int64)
    instantiation_dict::Dict{String, Int64} = Dict(
        csp.names[var] => instantiation[var] for var in 1:csp.n_variables)
    for x in 1:n
        println()
        for y in 1:n
            rank::Int64 = 0
            for var in 1:length(instantiation_dict)
                if instantiation_dict["position_" * string(var)] == square_from_coord(x, y, n)
                    rank = var
                    break
                end
            end
            print(string(rank) * " "^(3 - length(string(rank))))
        end
    end
end 

visualize_tour (generic function with 1 method)

In [26]:
tour = define_tour(6, "knight");
set_backtracking_params(
    tour,
    Dict(
        "matrix_storage"=>false,
        "time_limit"=>30.,
        "look_ahead_method"=> "FC",
        "variable_selection"=> "domain_size_random"
    )
)
@time b, i, n = solve!(tour)
visualize_tour(tour, i, 6)

  0.594759 seconds (407.57 k allocations: 49.143 MiB, 1.12% gc time)

21 14 31 10 1  12 
32 9  20 13 34 29 
15 22 33 30 11 2  
8  25 6  19 28 35 
5  16 23 26 3  18 
24 7  4  17 36 27 

### 2.3) Graph coloring

In [21]:
function generate_combinations(n::Int64, k::Int64)
    if k==1
        combs = Set([[i] for i in 1:n])
    else
        combs::Set{Array{Int64, 1}} = Set()
        for c in generate_combinations(n, k-1)
            for i in 1:n
                if !(i in c)
                    new_c = copy(c)
                    push!(new_c, i)
                    push!(combs, sort(new_c))
                end
            end
        end
        return combs
    end
end

function check_clique(adj, c)
    for u in c, v in c
        if (u != v) && adj[u, v] == 0
            return false
        end
    end
    return true
end

function find_kclique(n_vertices, edges, k)
    adj = zeros(Bool, n_vertices, n_vertices)
    for (u, v) in edges
        adj[u, v] = true
        adj[v, u] = 1
    end
    for c in generate_combinations(n_vertices, k)
       if check_clique(adj, c)
            return c
        end
    end
    return 0
end

import LightGraphs

function find_maxclique(n_vertices, edges)
    g = LightGraphs.SimpleGraph(n_vertices)
    for (u, v) in edges
        LightGraphs.add_edge!(g, u, v)
    end
    k = 0
    maxclique = []
    for c in LightGraphs.maximal_cliques(g)
        if length(c) >= k
            k = length(c)
            maxclique = c
        end
    end
    return maxclique
end

find_maxclique (generic function with 1 method)

In [22]:
function read_graph(path::String)
    n_vertices = 0
    edges = Array{Tuple{Int64, Int64}, 1}()
    open(path) do file
        for line in eachline(file)
            split_line = split(line)
            if line[1] == 'p'
                n_vertices = parse(Int64, split_line[3])
            elseif line[1] == 'e'
                u = parse(Int64, split_line[2])
                v = parse(Int64, split_line[3])
                push!(edges, (u, v))
            end
        end
    end
    return n_vertices, edges
end

read_graph (generic function with 1 method)

In [23]:
function compatibility_coloring(csp::CSP, var1::Int64, var2::Int64, value1::Int64, value2::Int64)
    return value1 != value2
end

function define_coloring(n_vertices::Int64, edges::Array{Tuple{Int64, Int64}, 1}, n_colors::Int64)
    csp = CSP()
    for u in 1:n_vertices
        name = "color_" * string(u)
        domain_size = n_colors
        add_variable!(csp, name, domain_size)
    end
    for (u, v) in edges
        add_constraint!(csp, u, v, compatibility_coloring)
        add_constraint!(csp, v, u, compatibility_coloring)
    end
    # Symmetry constraints
    maxclique = find_maxclique(n_vertices, edges)
    if length(maxclique) > n_colors
        maxclique = maxclique[1:n_colors]
    end
    print("Max clique size ", length(maxclique))
    for (i, u) in enumerate(maxclique)
        add_individual_constraint!(csp, u, (csp, u, c)->(c==i))
    end
    return csp
end

define_coloring (generic function with 1 method)

In [24]:
n_vertices, edges = read_graph("graphs/jean.col")
n_colors = 10
coloring = define_coloring(n_vertices, edges, n_colors);

Max clique size 10

In [25]:
set_backtracking_params(
    coloring,
    Dict(
        "matrix_storage"=>false,
        "time_limit"=>10.,
        "look_ahead_method"=> "FC",
        "variable_selection"=> "domain_size_random"
    )
)
@time b, i, n = solve!(coloring)

  0.002628 seconds (1.25 k allocations: 131.965 KiB)


(true, [1, 8, 9, 9, 10, 10, 2, 1, 3, 7  …  1, 6, 8, 1, 9, 1, 7, 9, 8, 1], 80)